In [1]:
from scipy.io.arff import loadarff
import pandas as pd

KeyboardInterrupt: 

#### Lectura de fichero

In [3]:
df_sick_train, meta_train = loadarff('sick/sick.fold.000000.train.arff')
df_sick_test, meta_test = loadarff('sick/sick.fold.000000.test.arff')

In [4]:
df_sick_train = pd.DataFrame(df_sick_train)
df_sick_test = pd.DataFrame(df_sick_test)
df_sick_train = df_sick_train.applymap(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
df_sick_test = df_sick_test.applymap(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)

C:\Users\xavid\AppData\Local\Temp\ipykernel_23736\2591950523.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_sick_train = df_sick_train.applymap(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
C:\Users\xavid\AppData\Local\Temp\ipykernel_23736\2591950523.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_sick_test = df_sick_test.applymap(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)


#### Data analysis

In [5]:
df_sick_train[df_sick_train['age'].isna()]

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_medication,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,...,TT4_measured,TT4,T4U_measured,T4U,FTI_measured,FTI,TBG_measured,TBG,referral_source,Class
1950,NaN,F,t,f,f,f,f,f,t,f,...,t,120.0,t,0.82,t,146.0,f,NaN,other,negative


- sex tiene 3 categorías
- las 2 de TBG fuera
- NaNs: regresión lineal
- Age quitar la fila con nan en Edad

In [17]:
for col in df_sick_train.columns:
    print(f'{col}: {df_sick_train[col].unique()}')

age: [0.07929515 0.14317181 0.17180617 0.16740088 0.13436123 0.12114537
 0.17621145 0.11674009 0.16960352 0.07268722 0.15638767 0.15198238
 0.16299559 0.04185022 0.05947137 0.11453744 0.07488987 0.15859031
 0.03744493 0.08370044 0.1123348  0.04845815 0.0969163  0.06387665
 0.07048458 0.13215859 0.06167401 0.06828194 0.1784141  0.10572687
 0.12334802 0.1277533  0.09251101 0.11013216 0.1938326  0.16079295
 0.14757709 0.04405286 0.02202643 0.0660793  0.17400881 0.13876652
 0.14096916 0.08590308 0.09030837 0.09471366 0.01321586 0.16519824
 0.11894273 0.13656388 0.08810573 0.0814978  0.10792952 0.14537445
 0.05286344 0.05066079 0.14977974 0.04625551 0.07709251 0.03964758
 0.12555066 0.10352423 0.12995595 0.03524229 0.09911894 0.18061674
 0.15418502 0.05506608 0.18942731 0.030837   0.05726872 0.00881057
 0.02863436 0.10132159 0.02643172 0.19603524 0.0154185  0.19823789
 0.         0.18281938 0.01101322 0.18502203 0.03303965 0.19162996
 0.18722467 0.01982379 0.02422907 0.00220264 0.00660793 0

In [7]:
columns_with_nans = []
for col in df_sick_train.columns:
    sum_nans = sum(df_sick_train[col].isna())
    percentage = sum_nans / len(df_sick_train) * 100
    print(f'{col}: {percentage}')
    if percentage > 0.1 and percentage < 99:
        columns_with_nans.append(col)

age: 0.029446407538280327
sex: 0.0
on_thyroxine: 0.0
query_on_thyroxine: 0.0
on_antithyroid_medication: 0.0
sick: 0.0
pregnant: 0.0
thyroid_surgery: 0.0
I131_treatment: 0.0
query_hypothyroid: 0.0
query_hyperthyroid: 0.0
lithium: 0.0
goitre: 0.0
tumor: 0.0
hypopituitary: 0.0
psych: 0.0
TSH_measured: 0.0
TSH: 9.74676089517079
T3_measured: 0.0
T3: 20.288574793875146
TT4_measured: 0.0
TT4: 6.154299175500589
T4U_measured: 0.0
T4U: 10.482921083627797
FTI_measured: 0.0
FTI: 10.424028268551238
TBG_measured: 0.0
TBG: 100.0
referral_source: 0.0
Class: 0.0


#### Drop rows age

In [8]:
from utils import drop_rows
df_sick_train = drop_rows(df_sick_train, ['age'])
df_sick_test = drop_rows(df_sick_test, ['age'])

In [9]:
from utils import drop_columns
df_sick_train = drop_columns(df_sick_train, ['TBG_measured', 'TBG'])
df_sick_test = drop_columns(df_sick_test, ['TBG_measured', 'TBG'])

In [10]:
from utils import min_max_scaler

df_sick_train, df_sick_test = min_max_scaler(df_sick_train,df_sick_test, ['age'])

In [11]:
from utils import one_hot_encoding

df_sick_train, df_sick_test = one_hot_encoding(df_sick_train,df_sick_test)

In [12]:
from utils import binary_encoding
df_sick_train, df_sick_test = binary_encoding(df_sick_train,df_sick_test)

In [13]:
from utils import fill_nans
df_sick_train, df_sick_test = fill_nans(df_sick_train, df_sick_test, columns_with_nans)

C:\Users\xavid\Documents\GitHub\IML-Work2\pythonProject\utils.py:89: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_test_nans = df_test[df_train[col].isna()]
C:\Users\xavid\Documents\GitHub\IML-Work2\pythonProject\utils.py:89: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_test_nans = df_test[df_train[col].isna()]
C:\Users\xavid\Documents\GitHub\IML-Work2\pythonProject\utils.py:89: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_test_nans = df_test[df_train[col].isna()]
C:\Users\xavid\Documents\GitHub\IML-Work2\pythonProject\utils.py:89: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_test_nans = df_test[df_train[col].isna()]
C:\Users\xavid\Documents\GitHub\IML-Work2\pythonProject\utils.py:89: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_test_nans = df_test[df_train[col].isna()]


In [16]:
df_sick_train, df_sick_test = min_max_scaler(df_sick_train,df_sick_test, columns_with_nans)